In [3]:
import pandas as pd
from collections import Counter

In [4]:
# Función para detectar ciclos de movimientos consecutivos.
def detect_cycles(group, n=3):
    movements = group["MOVEMENT"].values
    agents = group["AGENT.ID"].values

    # Inicializar contador de pares de agentes.
    pair_counts = Counter()

    # Generar patrones alternos de entrada y salida.
    for i in range(len(movements) - n + 1):
        # Verificar si la secuencia alterna (in, out, in, ...) o (out, in, out, ...) existe.
        in_out_pattern = all(movements[i+j] == ("in" if j % 2 == 0 else "out") for j in range(n))
        out_in_pattern = all(movements[i+j] == ("out" if j % 2 == 0 else "in") for j in range(n))


        if in_out_pattern or out_in_pattern:
            # Contar los pares únicos de agentes involucrados.
            cycle_agents = agents[i:i+n]
            unique_agents = sorted(set(cycle_agents))  # Eliminar duplicados y ordenar.
            
            # Generar los pares de agentes.
            for i in range(len(unique_agents)):
                for j in range(i + 1, len(unique_agents)):
                    pair_counts[(unique_agents[i], unique_agents[j])] += 1

    return pair_counts

In [5]:
# Cargar el archivo CSV.
data = pd.read_csv("data/ClientMovementsSN.csv", delimiter=";")

# Convertir la columna de fecha al formato adecuado.
data['DATE'] = pd.to_datetime(data['DATE'], format="%d/%m/%Y")

# Filtrar las filas donde el competidor 10 está involucrado.
filtered_data = data[
    (data['TRANSFEROR'] == "Competitor 10") | (data['RECEPIENT'] == "Competitor 10")
]

# Ordenar los datos.
filtered_data = filtered_data.sort_values(['ClientID', 'DATE'])

display(filtered_data)

,DATE,ClientID,MOVEMENT,BALANCE_IN,TRANSFEROR,BALANCE_OUT,RECEPIENT,AGENT.ID,STATUS,CHANNEL,aux,IN,OUT
1,2018-02-28,2,out,0,Competitor 9,0,Competitor 10,13087,Active,CHANNEL 5,1,0,1
2,2017-01-31,3,out,0,Competitor 9,17552962,Competitor 10,18024,Active,CHANNEL 5,1,0,1
4,2017-03-31,5,out,0,Competitor 9,29433839,Competitor 10,30677,Active,CHANNEL 5,1,0,1
5,2016-09-30,6,out,0,Competitor 9,6352446,Competitor 10,19040,Active,CHANNEL 5,1,0,1
6,2018-05-31,7,out,0,Competitor 9,11948542,Competitor 10,14204,Active,CHANNEL 5,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
749187,2020-01-31,689739,out,0,Competitor 9,18010023,Competitor 10,14586,Active,CHANNEL 5,1,0,1
749189,2019-03-31,689740,out,0,Competitor 9,1611736,Competitor 10,23885,Active,CHANNEL 5,1,0,1
749190,2017-09-30,689741,out,0,Competitor 9,565283,Competitor 10,5036,Active,CHANNEL 5,1,0,1
749194,2018-10-31,689745,out,0,Competitor 9,13770483,Competitor 10,16600,Active,CHANNEL 5,1,0,1


In [18]:
# Inicializar un contador global para los pares de agentes.
global_pair_counts = Counter()

# Listado para almacenar los resultados finales.
all_pairs = []

# Iterar sobre los datos agrupados por "ClientID".
for client_id, group in filtered_data.groupby("ClientID"):
    n = 2  # Iniciar en 2.
    while True:
        # Detectar ciclos con el valor de n.
        pair_counts = detect_cycles(group, n=n)

        # Salir si no se encontraron ciclos.
        if not pair_counts:
            break

        # Actualizar el contador global con los pares encontrados.
        global_pair_counts.update(pair_counts)

        # Almacenar los resultados con la intensidad (n).
        for pair, count in pair_counts.items():
            all_pairs.append({"AgentPair": pair, "ClientCount": count, "Intensity": n})

        # Incrementar n para la siguiente iteración.
        n += 1

# Convertir el listado de resultados en un DataFrame.
pairs_df = pd.DataFrame(all_pairs)

# Separar los pares en columnas "Agent1" y "Agent2".
pairs_df[["Agent1", "Agent2"]] = pd.DataFrame(pairs_df["AgentPair"].tolist(), index=pairs_df.index)

# Eliminar la columna redundante de tuplas.
pairs_df = pairs_df.drop(columns=["AgentPair"])

# Ordenar por número de clientes compartidos.
pairs_df = pairs_df.sort_values(by="ClientCount", ascending=False).reset_index(drop=True)

# Mostrar el resultado final.
display(pairs_df)

,ClientCount,Intensity,Agent1,Agent2
0,2,3,87,2553
1,2,3,87,2982
2,2,4,87,2553
3,2,4,87,2982
4,2,4,2553,2982
...,...,...,...,...
307,1,2,3556,16315
308,1,2,6350,13578
309,1,3,8410,30621
310,1,2,8410,30621


In [19]:
# Calcular estadísticas de los agentes.
agent_stats = (
    data.groupby("AGENT.ID")
    .agg(
        TotalIn=("BALANCE_IN", "sum"),
        TotalOut=("BALANCE_OUT", "sum"),
        Movements=("ClientID", "count"),
    )
    .reset_index()
)
agent_stats["NetBalance"] = agent_stats["TotalIn"] - agent_stats["TotalOut"]

# Mostrar las estadísticas de los agentes.
display(agent_stats)

,AGENT.ID,TotalIn,TotalOut,Movements,NetBalance
0,1,0,230657983,13,-230657983
1,2,2064711094,947219471,147,1117491623
2,3,0,2458069,1,-2458069
3,4,0,265623061,11,-265623061
4,5,0,61070458,24,-61070458
...,...,...,...,...,...
31286,31287,0,39392509,2,-39392509
31287,31288,0,13108566,1,-13108566
31288,31289,0,91244044,17,-91244044
31289,31290,0,668494,1,-668494


In [20]:
pairs_df.to_csv('/home/andres/Público/pairs_df.csv', index=False)
agent_stats.to_csv('/home/andres/Público/agent_stats.csv', index=False)

In [14]:
import networkx as nx

G = nx.Graph()
for _, row in pairs_df.iterrows():
    G.add_edge(row["Agent1"], row["Agent2"], weight=row["ClientCount"])

closeness = nx.closeness_centrality(G)
degree = dict(G.degree(weight="weight"))

In [15]:
display(closeness)

{np.int64(8410): 0.02195121951219512,
 np.int64(30621): 0.036585365853658534,
 np.int64(87): 0.04878048780487805,
 np.int64(2982): 0.039024390243902446,
 np.int64(2553): 0.039024390243902446,
 np.int64(21194): 0.012195121951219513,
 np.int64(23244): 0.012195121951219513,
 np.int64(4360): 0.012195121951219513,
 np.int64(14304): 0.012195121951219513,
 np.int64(4845): 0.016260162601626015,
 np.int64(24492): 0.024390243902439025,
 np.int64(14971): 0.012195121951219513,
 np.int64(15511): 0.012195121951219513,
 np.int64(21053): 0.02195121951219512,
 np.int64(9183): 0.012195121951219513,
 np.int64(10215): 0.012195121951219513,
 np.int64(19921): 0.012195121951219513,
 np.int64(24184): 0.012195121951219513,
 np.int64(17149): 0.016260162601626015,
 np.int64(15171): 0.012195121951219513,
 np.int64(18537): 0.012195121951219513,
 np.int64(3649): 0.024390243902439025,
 np.int64(3989): 0.024390243902439025,
 np.int64(4459): 0.024390243902439025,
 np.int64(1185): 0.024390243902439025,
 np.int64(6255):

In [16]:
display(degree)

{np.int64(8410): np.int64(5),
 np.int64(30621): np.int64(7),
 np.int64(87): np.int64(6),
 np.int64(2982): np.int64(4),
 np.int64(2553): np.int64(4),
 np.int64(21194): np.int64(1),
 np.int64(23244): np.int64(1),
 np.int64(4360): np.int64(1),
 np.int64(14304): np.int64(1),
 np.int64(4845): np.int64(1),
 np.int64(24492): np.int64(2),
 np.int64(14971): np.int64(1),
 np.int64(15511): np.int64(1),
 np.int64(21053): np.int64(1),
 np.int64(9183): np.int64(1),
 np.int64(10215): np.int64(1),
 np.int64(19921): np.int64(1),
 np.int64(24184): np.int64(1),
 np.int64(17149): np.int64(1),
 np.int64(15171): np.int64(1),
 np.int64(18537): np.int64(1),
 np.int64(3649): np.int64(2),
 np.int64(3989): np.int64(2),
 np.int64(4459): np.int64(2),
 np.int64(1185): np.int64(2),
 np.int64(6255): np.int64(2),
 np.int64(6499): np.int64(2),
 np.int64(15614): np.int64(1),
 np.int64(19119): np.int64(1),
 np.int64(1388): np.int64(2),
 np.int64(13737): np.int64(2),
 np.int64(28573): np.int64(2),
 np.int64(3351): np.int6